In [1]:
# !pip install selenium
# !pip install pyautogui

### Paso 0: Importar librerías

In [1]:
from selenium import webdriver #connect python with webbrowser-chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pyautogui as pag
import time

from collections import OrderedDict 

import pandas as pd

### Paso 1: Entrar a LinkedIn

In [5]:
# Entrar a la página de LinkedIn
options = Options()
options.add_argument("start-maximized") # Maximizar la pestaña de Chrome
options.add_argument("--disable-extensions") # Deshabilitar extensiones
driver = webdriver.Chrome("chromedriver.exe", options=options) # Especificar la ruta del Chrome Driver (en este caso está ubicado en la misma ruta del código)
driver.get("https://linkedin.com") 

### Paso 2: Loguearte

In [6]:
# Añadir credenciales de la cuenta
username_ = '' # tu username o correo
password_ = '' # tu password


# Ubicar campos username y password en la página web de Instagram
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='session_key']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='session_password']")))

# Ingresar username y password
username.clear()
username.send_keys(username_)
password.clear()
password.send_keys(password_)

time.sleep(2)

# Ubicar al botón de 'iniciar sesión' y hacer clic en él
button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

### Paso 3: Entrar a la publicación e identificar a los que reaccionaron

In [91]:
post = 'https://www.linkedin.com/feed/update/urn:li:activity:6889942708153065472/'

# Acceder a la página de la publicación
driver.get(post)
time.sleep(5)

# Encontrar el elemento de reacciones de la publicación y desplegar
reacciones = driver.find_element_by_xpath("//span[@class='social-details-social-counts__social-proof-container']")
reacciones.click()
time.sleep(3)

# Como las reacciones se despliegan en un pop-up, la forma de scrollear es diferente que si fuera la página normal
pop_up_window = WebDriverWait(driver, 2).until(EC.element_to_be_clickable(
    (By.XPATH, "//div[@class='artdeco-modal__content social-details-reactors-modal__content ember-view']")))

# Scrollear todas las reacciones para que todos los perfiles que reaccionaron aparezcan en el html del pop-up
t_end = time.time() + 30 # ~ 2 sec por 18 reacciones
while time.time() < t_end:
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].scrollTop + arguments[0].offsetHeight;', pop_up_window)

### Paso 4: Obtener dataframe con nombre, url, grado de conexión y headline de la comunidad

In [118]:
# Obtener todos los elementos "li" (la mayoría de elementos "li" contienen el nombre, url, grado de conexión y headline de los que reaccionaron)
elems_li = driver.find_elements_by_tag_name("li")

# Obtener el número de reacciones de la publicación
reacciones = driver.find_element_by_xpath("//span[@class='social-details-social-counts__social-proof-fallback-number']")
num_reacc = int(reacciones.get_attribute('innerHTML'))

# Como algunos elementos "li" no contienen las características de las personas que reaccionaron, se deben eliminar
num_li_elim = (len(elems_li) - num_reacc)*-1
elems_li = elems_li[:num_li_elim]

links = [] # Lista donnde se almacenará los links de los perfiles
df = pd.DataFrame(columns = ["nombre", "url", "grado_conexion", "headline"], dtype=object) # Dataframe que contendrá las característicase las personas que reaccionaron

for i in elems_li:
    # Obtener el link
    persona = i.find_elements_by_tag_name("a")
    elems_href = [j.get_attribute('href') for j in persona]
    if elems_href:
        link = elems_href[0] # El primer elemento "href" contiene el link de los perfiles
    links.append(link)
    
    # Obtener nombre, headline y grado de conexión (1st, 2nd o 3rd)
    nombre = [l.find_elements_by_xpath("//div//span[@dir='ltr']") for l in persona]
    grado_conexion = [m.find_elements_by_xpath("//div//span[@class='a11y-text']") for m in persona]
    headline = [n.find_elements_by_xpath("//div//div[@class='artdeco-entity-lockup__caption ember-view']") for n in persona]
    
    nombre = [o[len(links)-1].get_attribute('innerHTML') for o in nombre]
    grado_conexion = [p[len(links)].get_attribute('innerHTML') for p in grado_conexion]
    headline = [q[len(links)-1].get_attribute('innerHTML') for q in headline]

    # Consolidar en el dataframe todas las características de los perfiles
    df.loc[df.shape[0]] = [nombre[0], link, grado_conexion[0], headline[0]] # 0 porque son listas
df

,nombre,url,grado_conexion,headline
0,Rosana Ferrero,https://www.linkedin.com/in/rosana-ferrero-484...,1st degree connection,"\n Científica de datos, Direc..."
1,David Vega,https://www.linkedin.com/in/davidvegalugo?mini...,1st degree connection,\n Finance &amp; Strategy | M...
2,Robin Estrada Ventura,https://www.linkedin.com/in/robin-estrada?mini...,1st degree connection,\n Catman &amp; New Channels ...
3,Nikol Cotrina Araujo,https://www.linkedin.com/in/nikolcotrina?miniP...,1st degree connection,\n Project Manager\n ...
4,Nicolle Ximena Rosas López,https://www.linkedin.com/in/nicolle-ximena-ros...,1st degree connection,\n Estudiante de Gestión en P...
...,...,...,...,...
291,Cristina Rivera,https://www.linkedin.com/in/ACoAACgVmqsBufIiKA...,2nd degree connection,\n Risk Specialist at BCP | E...
292,Lucía Ivett Amambal del Águila,https://www.linkedin.com/in/ACoAAC-i1hcBivgqVB...,2nd degree connection,\n Estudiante de 9no ciclo de...
293,Jordany Farroñay Blanco,https://www.linkedin.com/in/ACoAABpFSggBYUmRCO...,2nd degree connection,\n Treasury and Capital Marke...
294,Bryan Antonio Alvarado Villalobos,https://www.linkedin.com/in/ACoAACVzgVsBiagNxN...,2nd degree connection,\n Scrum | Google Cloud Platf...


### Paso 5: Limpiar tabla y quedarse con las personas de 2do y 3er grado de conexión

In [124]:
# Limpiar el headline (quitar los espacios y "\n")
for index in df.index:
    df.loc[index,'headline'] = df.loc[index,'headline'][19:-17]

# Reemplazar "&nbsp;" por "3rd degree connection" porque para este grado así se presenta en el html
df["grado_conexion"] = df["grado_conexion"].replace({"&nbsp;": "3rd degree connection"})

# Quedarse con los de 2do y 3er grado de conexión
df = df.loc[df['grado_conexion'] != "1st degree connection"]
df

C:\Users\SANDRO\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,nombre,url,grado_conexion,headline
218,Jannina Danitza Romero Mori,https://www.linkedin.com/in/ACoAADOqTwoB1W_WtJ...,2nd degree connection,Ingeniería Industrial | UNMSM | PSM-PMO| IEEE ...
219,Sebastián Carbonel Solano,https://www.linkedin.com/in/ACoAAChooccB-wS2IZ...,3rd degree connection,Estudiante de Publicidad
220,Thaís Bello Ojeda,https://www.linkedin.com/in/ACoAACc2X7YBN6YBuj...,2nd degree connection,Senior It Talent Acquisition Specialist to Cos...
221,José Rodrigo Tello Rojas,https://www.linkedin.com/in/ACoAACpiTO8BGDCYiB...,2nd degree connection,Asistente comercial en CADENT S.A.C.
222,Adriana Flores Huaman,https://www.linkedin.com/in/ACoAADNGwcUBZVLitC...,2nd degree connection,Atención al Cliente / Estudiante de Contabilid...
...,...,...,...,...
291,Cristina Rivera,https://www.linkedin.com/in/ACoAACgVmqsBufIiKA...,2nd degree connection,Risk Specialist at BCP | Economics graduate | ...
292,Lucía Ivett Amambal del Águila,https://www.linkedin.com/in/ACoAAC-i1hcBivgqVB...,2nd degree connection,Estudiante de 9no ciclo de Economía
293,Jordany Farroñay Blanco,https://www.linkedin.com/in/ACoAABpFSggBYUmRCO...,2nd degree connection,Treasury and Capital Markets Jr Consultant en ...
294,Bryan Antonio Alvarado Villalobos,https://www.linkedin.com/in/ACoAACVzgVsBiagNxN...,2nd degree connection,Scrum | Google Cloud Platform | Spring | Dropw...


### Paso 6, 7, 8 y 9: Enviar invitaciones masivas y personalizadas a tu audiencia 

In [1]:
 for index in df.index:
    # Paso 6: Ingresar a la url del perfil de la persona
    url = df.loc[index,'url']
    driver.get(url)
    time.sleep(2)
    
    # Paso 7: Conectar con la persona (2nd y 3rd grado)
    conectar = driver.find_elements_by_xpath("//section//div//div//div//button//span[@class='artdeco-button__text']")

    connect = [i for i in conectar if i.get_attribute('innerHTML') == "\n    Connect\n"]
    if connect: # Si inicialmente le aparece "Connect"
        driver.execute_script("arguments[0].click();", connect[0])
    else: # Si la persona tiene perfil de creador y le aparece "Follow" en vez de "Connect"
        buttons = driver.find_elements_by_tag_name("button")
        more_buttons = [btn for btn in buttons if btn.text == "More"]
        location = more_buttons[0].location 
        pag.click(list(location.values())[0]+150, list(location.values())[1]+280)
        pag.move(0, 150) # bajar el mouse 150 pixeles desde el button "More"
        pag.click()
        connect = driver.find_element_by_xpath("//button[@aria-label='Connect']")
        driver.execute_script("arguments[0].click();", connect)
    
    # Paso 8: Agregar una nota (mensaje) personalizada al invite
    ## Paso 8.1: Permitir agregar una nota al invite
    add_note = driver.find_element_by_xpath("//button[@aria-label='Add a note']")
    driver.execute_script("arguments[0].click();", add_note)

    ## Paso 8.2: Formar y enviar el mensaje en la nota
    enviar_mensaje = driver.find_element_by_xpath("//textarea[@name='message']")    
    
    nombre = df.loc[index,'nombre'].split()[0] # Primera palabra de la persona
    
    mensaje = ("Hola, " + nombre + "!, buen día! Sería un gusto estar en contacto. Mis ramas de interés van por tecnología, startups e inversiones. " +
               "Si puedo agregar valor en algo, encantado! Te invito a que podamos crecer en comunidad :D")
    enviar_mensaje.send_keys(mensaje)
    time.sleep(1)
    
    # Paso 9: Enviar el invite con la nota personalizada
    pag.click(1236,715)